In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk

In [2]:
pip install accelerate -U

  Using cached accelerate-0.29.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.29.2-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.1
    Uninstalling accelerate-0.29.1:
      Successfully uninstalled accelerate-0.29.1
  Rolling back uninstall of accelerate
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-config.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-udi26dj8\accelerate-config.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-estimate-memory.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-udi26dj8\accelerate-estimate-memory.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-launch.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-udi26dj8\accelerate-launch.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate.exe
   from C:\Users\WinTo\AppData\Loc

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\accelerate.exe' -> 'c:\\Python311\\Scripts\\accelerate.exe.deleteme'



In [2]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

In [3]:
# Tokenizer and model setup
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125M')
tokenizer.pad_token = tokenizer.eos_token

In [5]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = inputs
        self.outputs = outputs
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        
        input_encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
        output_encodings = tokenizer(output_text, padding="max_length", truncation=True, max_length=self.max_length)
        
        return {
            "input_ids": torch.tensor(input_encodings['input_ids']),
            "attention_mask": torch.tensor(input_encodings['attention_mask']),
            "labels": torch.tensor(output_encodings['input_ids'])
        }
    

# Load and preprocess data
df = pd.read_json('Psychology-10K.json')
inputs = df['input'].tolist()
outputs = df['output'].tolist()
dataset = TextDataset(tokenizer, inputs, outputs)

In [9]:
# Training arguments and setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5, 
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


In [12]:
# Train the model
trainer.train()

  0%|          | 0/7880 [00:00<?, ?it/s]

{'loss': 1.7832, 'grad_norm': 233393.8125, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 1.7063, 'grad_norm': 257102.78125, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 1.7028, 'grad_norm': 105453.71875, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 1.3977, 'grad_norm': 364785.75, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 1.4128, 'grad_norm': 18980.697265625, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 1.3583, 'grad_norm': 63940.2734375, 'learning_rate': 6e-06, 'epoch': 0.03}
{'loss': 1.2922, 'grad_norm': 24337.361328125, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 1.2159, 'grad_norm': 50195.70703125, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.04}
{'loss': 1.2432, 'grad_norm': 24527.345703125, 'learning_rate': 9e-06, 'epoch': 0.05}
{'loss': 1.2736, 'grad_norm': 14798.7236328125, 'learning_rate': 1e-05, 'epoch': 0.05}
{'loss': 1.2911, 'grad_norm': 12435.9921875, 'learning_rate': 1.100000000000

TrainOutput(global_step=7880, training_loss=11.376474951971606, metrics={'train_runtime': 4643.2896, 'train_samples_per_second': 8.482, 'train_steps_per_second': 1.697, 'train_loss': 11.376474951971606, 'epoch': 4.0})

In [37]:
def predict_response(input_text, model, tokenizer, max_length=512):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(model.device)
    model.eval()  # Set the model to evaluation mode
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=50,
    temperature=0.5 ,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id)
    predicted_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return predicted_text     

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel depressed"

response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

Predicted Response: i feel depressed, these all in. and's your the ways on both is that just at, as needs this be you to activities can more such levels of. career and
